In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, RandomSampler
import random
from transformers import AdamW, AutoTokenizer, AutoModel, DataCollatorWithPadding, AutoConfig
from torch.utils.data import Dataset, DataLoader, TensorDataset, RandomSampler
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.nn import Parameter
import math
from torch.optim import Adam, lr_scheduler
from sklearn.model_selection import KFold
import urllib.request
from typing import List
from functools import partial
from sklearn.model_selection import StratifiedShuffleSplit, GroupKFold, KFold
from torch.cuda.amp import autocast, GradScaler
from transformers.models.deberta.modeling_deberta import ContextPooler
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import scipy.sparse as sp
from math import log
import string # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train = pd.read_csv('feedback-prize-effectiveness/train.csv')
test = pd.read_csv('feedback-prize-effectiveness/test.csv')

In [3]:
TRAIN_DIR = "feedback-prize-effectiveness/train"
TEST_DIR = "feedback-prize-effectiveness/test"

In [4]:
def train_get_essay(essay_id):
    essay_path = os.path.join(TRAIN_DIR, f"{essay_id}.txt")
    essay_text = open(essay_path, 'r').read()
    return essay_text
def test_get_essay(essay_id):
    essay_path = os.path.join(TEST_DIR, f"{essay_id}.txt")
    essay_text = open(essay_path, 'r').read()
    return essay_text

In [5]:
train['essay_text'] = train['essay_id'].apply(train_get_essay)

In [6]:
test['essay_text'] = test['essay_id'].apply(test_get_essay)

In [7]:
train

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,essay_text
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate,Some people may ask multiple people for advice...
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate,Some people may ask multiple people for advice...
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate,Some people may ask multiple people for advice...
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective,Some people may ask multiple people for advice...


In [8]:
train['text'] = train['discourse_text'] + " " + train['discourse_type'] + " " + train['essay_text']
test['text'] = test['discourse_text'] + " " + test['discourse_type'] + " " + test['essay_text']

In [9]:
word_embeddings_dim = 300
word_vector_map = {}

In [10]:
train_lst = []
test_lst = []
for x in train['text']:
    train_lst.append(x)
for x in test['text']:
    test_lst.append(x)

In [47]:
def build_graph(shuffle_doc_words_lst):
    # build vocab
    word_freq = {}
    word_set = set()
    for doc_words in tqdm(shuffle_doc_words_list):
        words = doc_words.split()
        for word in words:
            word_set.add(word)
            if word in word_freq:
                word_freq[word] += 1
            else:
                word_freq[word] = 1

    vocab = list(word_set)
    vocab_size = len(vocab)

    word_doc_list = {}

    for i in tqdm(range(len(shuffle_doc_words_list))):
        doc_words = shuffle_doc_words_list[i]
        words = doc_words.split()
        appeared = set()
        for word in words:
            if word in appeared:
                continue
            if word in word_doc_list:
                doc_list = word_doc_list[word]
                doc_list.append(i)
                word_doc_list[word] = doc_list
            else:
                word_doc_list[word] = [i]
            appeared.add(word)

    word_doc_freq = {}
    for word, doc_list in word_doc_list.items():
        word_doc_freq[word] = len(doc_list)

    word_id_map = {}
    for i in range(vocab_size):
        word_id_map[vocab[i]] = i

    len(word_doc_freq)

    row_x = []
    col_x = []
    data_x = []
    for i in tqdm(range(len(shuffle_doc_words_list))):
        doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
        doc_words = shuffle_doc_words_list[i]
        words = doc_words.split()
        doc_len = len(words)
        for word in words:
            if word in word_vector_map:
                word_vector = word_vector_map[word]
                # print(doc_vec)
                # print(np.array(word_vector))
                doc_vec = doc_vec + np.array(word_vector)

        for j in range(word_embeddings_dim):
            row_x.append(i)
            col_x.append(j)
            # np.random.uniform(-0.25, 0.25)
            data_x.append(doc_vec[j] / doc_len)  # doc_vec[j]/ doc_len

    x = sp.csr_matrix((data_x, (row_x, col_x)), shape=(
        len(shuffle_doc_words_list), word_embeddings_dim))

    # y = []
    # for i in range(len(train['text'])):
    #     doc_meta = train['text'][i]
    #     temp = doc_meta.split('\t')
    #     label = temp[2]
    #     one_hot = [0 for l in range(len(label_list))]
    #     label_index = label_list.index(label)
    #     one_hot[label_index] = 1
    #     y.append(one_hot)
    # y = np.array(y)
    # print(y)

    window_size = 20
    windows = []

    for doc_words in tqdm(shuffle_doc_words_list):
        words = doc_words.split()
        length = len(words)
        if length <= window_size:
            windows.append(words)
        else:
            # print(length, length - window_size + 1)
            for j in range(length - window_size + 1):
                window = words[j: j + window_size]
                windows.append(window)
                # print(window)

    word_window_freq = {}
    for window in tqdm(windows):
        appeared = set()
        for i in range(len(window)):
            if window[i] in appeared:
                continue
            if window[i] in word_window_freq:
                word_window_freq[window[i]] += 1
            else:
                word_window_freq[window[i]] = 1
            appeared.add(window[i])

    word_pair_count = {}
    for window in tqdm(windows):
        for i in range(1, len(window)):
            for j in range(0, i):
                word_i = window[i]
                word_i_id = word_id_map[word_i]
                word_j = window[j]
                word_j_id = word_id_map[word_j]
                if word_i_id == word_j_id:
                    continue
                word_pair_str = str(word_i_id) + ',' + str(word_j_id)
                if word_pair_str in word_pair_count:
                    word_pair_count[word_pair_str] += 1
                else:
                    word_pair_count[word_pair_str] = 1
                # two orders
                word_pair_str = str(word_j_id) + ',' + str(word_i_id)
                if word_pair_str in word_pair_count:
                    word_pair_count[word_pair_str] += 1
                else:
                    word_pair_count[word_pair_str] = 1

    row = []
    col = []
    weight = []
    # train_size = len(train['text'])
    train_size = len(shuffle_doc_words_list)
    # pmi as weights

    num_window = len(windows)

    for key in word_pair_count:
        temp = key.split(',')
        i = int(temp[0])
        j = int(temp[1])
        count = word_pair_count[key]
        word_freq_i = word_window_freq[vocab[i]]
        word_freq_j = word_window_freq[vocab[j]]
        pmi = log((1.0 * count / num_window) /
                  (1.0 * word_freq_i * word_freq_j/(num_window * num_window)))
        if pmi <= 0:
            continue
        row.append(train_size + i)
        col.append(train_size + j)
        weight.append(pmi)

    doc_word_freq = {}

    for doc_id in tqdm(range(len(shuffle_doc_words_list))):
        doc_words = shuffle_doc_words_list[doc_id]
        words = doc_words.split()
        for word in words:
            word_id = word_id_map[word]
            doc_word_str = str(doc_id) + ',' + str(word_id)
            if doc_word_str in doc_word_freq:
                doc_word_freq[doc_word_str] += 1
            else:
                doc_word_freq[doc_word_str] = 1

    for i in tqdm(range(len(shuffle_doc_words_list))):
        doc_words = shuffle_doc_words_list[i]
        words = doc_words.split()
        doc_word_set = set()
        for word in words:
            if word in doc_word_set:
                continue
            j = word_id_map[word]
            key = str(i) + ',' + str(j)
            freq = doc_word_freq[key]
            if i < train_size:
                row.append(i)
            else:
                row.append(i + vocab_size)
            col.append(train_size + j)
            idf = log(1.0 * len(shuffle_doc_words_list) /
                      word_doc_freq[vocab[j]])
            weight.append(freq * idf)
            doc_word_set.add(word)

    node_size = train_size + vocab_size
    adj = sp.csr_matrix(
        (weight, (row, col)), shape=(node_size, node_size))
    return adj

In [ ]:
adj = build_graph(train_lst)

In [ ]:
np.savez("train_adj", data=adj.data, indices=adj.indices,
             indptr=adj.indptr, shape=adj.shape)

In [48]:
adj2 = build_graph(test_lst)

100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 2506.76it/s]


In [44]:
np.savez("test_adj", data=adj2.data, indices=adj2.indices,
             indptr=adj2.indptr, shape=adj2.shape)